# Cel zadania i problem biznesowy


# Preprocessing

Nasz zbiór danych w postaci .csv zawierał aż 33 kolumny, co wiązało się z faktem że nasz preprocessing był dość obszerny. Do każdej kolumny staraliśmy się podejść indywidualnie i odpowiednio przetransformować jej wartości, aby póżniej algorytm uczenia maszynowego działał poprawnie. 

W przypadku kolumn kategorycznych zazwyczaj decydowaliśmy się na użycie algorytmu OneHotEncoding z pakietu scikit-learn. Jest to dość intuicyjna metoda, dzięki której nie tracimy żadnych informacji. Dodaje on do zbioru danych dodatkowe kolumny (ich liczność jest odpowiednia liczbą kategorii), a następnie daje wartość 1 tam, gdzie jest odpowiednia kategoria. Sprawia to, że informacje te są zakodowane w postaci liczbowej. Niestety sprawia to, że dodajemy wiele dodatkowych kolumn, co negatywnie wpływa na interpretowalność modelu zbudowanego na tej podstawie. Przykładowymi kolumnami kategorycznymi, na których zastosowaliśmy ten algorytm to: 'TopThreeAmericanName', 'WheelTypeID','Nationality', itd... 

Kolejnym ważnym etapem preprocessingu było zastosowanie normalizacji danych. Zważając na fakt, że budowaliśmy modele które są czułe na skale zmiennych (np. SVM) musieliśmy wszystkie wartości liczbowe w jakiś sposób przeskalować, aby były między sobą 'porównywalne'. Zastosowaliśmy funkcję StandardScaler, która odejmuje od każdej wartości średnią arytmetyczną (mean), a następnie dzieli przez wariancję. Przykładowymi kolumnami objętymi tym algorytmem są: 'VehOdo','VehBCost', 'WarrantyCost', itd ... 


# Model



